## Feature Engineering & Data cleaning

This notebook covers features engineering, data cleaning:

According to initial data analysis I decided to a

In [1]:
import numpy as np

In [2]:
# The code was removed by Watson Studio for sharing.

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,fuel1,fuel2,fuel3,...,owner,source,url,geolocation_source,year_of_capacity_data,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,estimated_generation_gwh
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.00,32.3220,65.1190,Hydro,NaN,NaN,...,NaN,GEODB,http://globalenergyobservatory.org,GEODB,2017.0,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.00,34.5560,69.4787,Hydro,NaN,NaN,...,NaN,GEODB,http://globalenergyobservatory.org,GEODB,2017.0,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.00,34.6410,69.7170,Hydro,NaN,NaN,...,NaN,GEODB,http://globalenergyobservatory.org,GEODB,2017.0,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,Nangarhar (Darunta) Hydroelectric Power Plant ...,GEODB0040536,11.55,34.4847,70.3633,Hydro,NaN,NaN,...,NaN,GEODB,http://globalenergyobservatory.org,GEODB,2017.0,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,Northwest Kabul Power Plant Afghanistan,GEODB0040540,42.00,34.5638,69.1134,Gas,NaN,NaN,...,NaN,GEODB,http://globalenergyobservatory.org,GEODB,2017.0,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.drop(['country_long', 'url', 'geolocation_source', 'name', 'gppd_idnr', 'source', 'fuel2', 'fuel3', 'fuel4', 'latitude', 'longitude', 'year_of_capacity_data', 'owner', 'commissioning_year'], axis=1)

First for all let's convert data to appropriate format

In [4]:
df['country'] = df['country'].astype('category')
df['capacity_mw'] = df['capacity_mw'].astype('float32')
df['fuel1'] = df['fuel1'].astype('category')
df['generation_gwh_2013'] = df['generation_gwh_2013'].astype('float32')
df['generation_gwh_2014'] = df['generation_gwh_2014'].astype('float32')
df['generation_gwh_2015'] = df['generation_gwh_2015'].astype('float32')
df['generation_gwh_2016'] = df['generation_gwh_2016'].astype('float32')
df['estimated_generation_gwh'] = df['estimated_generation_gwh'].astype('float32')

In [5]:
from sklearn.preprocessing import StandardScaler

def mean_generated_energy(x):
    values = []
    for col in ['generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015', 'generation_gwh_2016', 'estimated_generation_gwh']:
        if not np.isnan(x[col]):
            values.append(x[col])
    if len(values) != 0:
        return np.mean(values)  
    else:
        return 0

df['mean_gen_gwh'] = df.apply(mean_generated_energy, axis=1)

In [6]:
df = df.iloc[np.where(df['fuel1'].notna())[0],:]
df = df.iloc[np.where(df['country'].notna())[0],:]
df = df[df['mean_gen_gwh'] != 0]

In [7]:
df = df.drop(['generation_gwh_2014', 'generation_gwh_2015', 'generation_gwh_2016', 'estimated_generation_gwh', 'generation_gwh_2013'], axis=1)

In [19]:
df['capacity_mw'] = StandardScaler(copy=False).fit_transform(df['capacity_mw'][:, np.newaxis])

In [8]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
country_features = encoder.fit_transform(df['country'].astype('category'))

In [9]:
encoder = LabelBinarizer()
fuel1_features = encoder.fit_transform(df['fuel1'].astype('category'))

In [10]:
df = df.drop(['fuel1', 'country'],axis=1)

In [11]:
dataset_for_train = pd.concat([df.reset_index(), pd.DataFrame(fuel1_features).reset_index(), pd.DataFrame(country_features).reset_index()], axis=1).drop('index', axis=1)
dataset_for_train.columns = list(map(lambda x: str(x), dataset_for_train.columns))

In [12]:
#dataset_for_train.dropna(x, axis)
generated_cols = list(map(lambda x: str(x), np.arange(160)))
dataset_for_train = dataset_for_train.iloc[np.where(dataset_for_train[generated_cols[0]].notna())[0],:]

In [13]:
dataset_for_train.reset_index().to_csv('data_for_train.csv')

In [14]:
import os
os.listdir()

['data_for_train.csv']

In [15]:
# The code was removed by Watson Studio for sharing.

In [16]:
cos.upload_file(Filename='data_for_train.csv', Bucket=credentials['BUCKET'], Key='data_for_train.csv')
print('Success')

Success


In [17]:
dataset_for_train.head()

,capacity_mw,mean_gen_gwh,0,1,2,3,4,5,6,7,...,125,126,127,128,129,130,131,132,133,134
0,27.0,89.132072,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
0,27.0,89.132072,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,500.0,1650.593994,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,500.0,1650.593994,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,600.0,1980.712769,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
